In [1]:
from transformers import BertModel, BertTokenizer, AdamWeightDecay, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from textwrap import wrap

c:\Users\ISAIAS_QUINTERO\Desktop\Trabajo Practica\venv\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\ISAIAS_QUINTERO\Desktop\Trabajo Practica\venv\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Inicialization of the BERT model
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 16
DATASET_PATH = "/content/datos.xlsx"
NCLASS = 6

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
# Cargar dataset

df = pd.read_parquet(r"C:\Users\ISAIAS_QUINTERO\Desktop\Trabajo Practica\datos_modelo.parquet")
df.head()


,SEXO,EDAD,GRUPO,ESPECIALIDAD_MEDICA,SUBJETIVO,OBJETIVO,Concatenada,Label_Grupo,Concatenado1
0,Femenino,38,Otros Trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","[empezar, episodio, tom, pastilla, dormir, ten...",0,empezar episodio tom pastilla dormir tener pro...
1,Masculino,22,T. Externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","[sentido, trabajar, concentrar yo, cosa, vidat...",2,sentido trabajar concentrar yo cosa vidatodo d...
2,Masculino,9,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"[madre, venir, acompañamiento, emocional, sepr...",0,madre venir acompañamiento emocional sepracion...
3,Masculino,28,Otros Trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","[cosa, problema, empresa, ocasión, saludar, sa...",0,cosa problema empresa ocasión saludar saludo e...
4,Femenino,8,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","[madre, manifestado, conducta, palabrasella, r...",0,madre manifestado conducta palabrasella relaci...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   SEXO                 1000 non-null   object
 1   EDAD                 1000 non-null   int64 
 2   GRUPO                1000 non-null   object
 3   ESPECIALIDAD_MEDICA  1000 non-null   object
 4   SUBJETIVO            956 non-null    object
 5   OBJETIVO             984 non-null    object
 6   Concatenada          1000 non-null   object
 7   Label_Grupo          1000 non-null   int32 
 8   Concatenado1         1000 non-null   object
dtypes: int32(1), int64(1), object(7)
memory usage: 66.5+ KB


In [5]:
# Eliminamos filas con valores nulos
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 943 entries, 0 to 999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   SEXO                 943 non-null    object
 1   EDAD                 943 non-null    int64 
 2   GRUPO                943 non-null    object
 3   ESPECIALIDAD_MEDICA  943 non-null    object
 4   SUBJETIVO            943 non-null    object
 5   OBJETIVO             943 non-null    object
 6   Concatenada          943 non-null    object
 7   Label_Grupo          943 non-null    int32 
 8   Concatenado1         943 non-null    object
dtypes: int32(1), int64(1), object(7)
memory usage: 70.0+ KB


In [6]:
df.head(10)

,SEXO,EDAD,GRUPO,ESPECIALIDAD_MEDICA,SUBJETIVO,OBJETIVO,Concatenada,Label_Grupo,Concatenado1
0,Femenino,38,Otros Trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","[empezar, episodio, tom, pastilla, dormir, ten...",0,empezar episodio tom pastilla dormir tener pro...
1,Masculino,22,T. Externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","[sentido, trabajar, concentrar yo, cosa, vidat...",2,sentido trabajar concentrar yo cosa vidatodo d...
3,Masculino,28,Otros Trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","[cosa, problema, empresa, ocasión, saludar, sa...",0,cosa problema empresa ocasión saludar saludo e...
4,Femenino,8,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","[madre, manifestado, conducta, palabrasella, r...",0,madre manifestado conducta palabrasella relaci...
5,Masculino,30,T. Externalizantes,PSICOLOGÍA,"Paciente refiere ""yo creo como todo comienza d...","Paciente quien evalúo por primera vez, pacien...","[comenzar, crianza, mama, diagnostico, tab, se...",2,comenzar crianza mama diagnostico tab sentir c...
6,Femenino,60,Otros Trastornos,PSICOLOGÍA,"Paciente refiere ""hace 3 años me separe, hace ...","Paciente quien evalúo por primera vez, pacient...","[año, separar, año, hija, tambin, trabajar, fu...",0,año separar año hija tambin trabajar fuerame s...
7,Masculino,57,Otros Trastornos,PSICOLOGÍA,"Paciente refiere ""Yo he sido muy introvertido,...","Paciente quien evalúo por primera vez, pacient...","[introvertido, expresar, sentimiento, tragar, ...",0,introvertido expresar sentimiento tragar indep...
8,Femenino,25,T. Externalizantes,PSICOLOGÍA,"Hace un mes empece a sentir ansiedad, tuve com...","Paciente quien evalúo por primera vez, pacient...","[mes, empece, sentir, ansiedad, tener, semana,...",2,mes empece sentir ansiedad tener semana tenia ...
9,Femenino,40,Otros Trastornos,PSICOLOGÍA,"Paciente refiere ""en los 15 de mi prima yo le ...","Paciente quien evalúo por primera vez, la cons...","[prima, llevar, comida, tiro, marco, tambin, r...",0,prima llevar comida tiro marco tambin rabia co...
10,Masculino,31,Otros Trastornos,PSICOLOGÍA,"Paciente refiere:\n""Luego de ruptura con mi ex...","Paciente alerta y consciente, orientado en per...","[refiereluego, ruptura, exparejo, detonar, pro...",0,refiereluego ruptura exparejo detonar problema...


In [7]:
from transformers import BertTokenizer, BertModel

# TOKENIZACION

PRE_TRAINED_MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

c:\Users\ISAIAS_QUINTERO\Desktop\Trabajo Practica\venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# Ejemplos de tokenización

sample_text = "Hola soy un texto de ejemplo para la tokenización."
tokens = tokenizer.tokenize(sample_text)
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"Texto original: {sample_text}")
print(f"Tokens: {tokens}")
print(f"IDs de tokens: {tokens_ids}")

Texto original: Hola soy un texto de ejemplo para la tokenización.
Tokens: ['Ho', '##la', 'soy', 'un', 'texto', 'de', 'ejemplo', 'para', 'la', 'tok', '##eni', '##zación', '.']
IDs de tokens: [20220, 10330, 103559, 10119, 27888, 10104, 20223, 10220, 10109, 18436, 18687, 23700, 119]


In [9]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.4.1+cpu
False


In [10]:
# Instala PyTorch si no está instalado
%pip install torch

# Codificacion para introducir a BERT

encoding = tokenizer.encode_plus(
    sample_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors="pt",
)

Note: you may need to restart the kernel to use updated packages.


In [11]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [12]:
print(tokenizer.convert_ids_to_tokens(encoding["input_ids"][0]))
print(encoding["input_ids"][0])
print(encoding["attention_mask"][0])

['[CLS]', 'Ho', '##la', 'soy', 'un', 'texto', 'de', 'ejemplo', 'para', 'la', 'tok', '##eni', '##zación', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [13]:
# CREACION DEL DATASET

class CustomDataset(Dataset):
    def __init__(self, text, labels, tokenizer, max_len):
        self.text = text
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
      return len(self.text)

    def __getitem__(self, item):
      text = str(self.text[item])
      label = self.labels[item]
      encoding = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          truncation=True,
          return_attention_mask=True,
          return_tensors="pt",
      )
      return {"text": text,
                "input_ids": encoding["input_ids"].flatten(),
                "attention_mask": encoding["attention_mask"].flatten(),
                "labels": torch.tensor(label, dtype=torch.long)}


In [14]:
# Data Loader:

def data_loader(df, tokenizer, max_len, batch_size):
    dataset = CustomDataset(
        text = df["Concatenado1"].to_numpy(),
        labels = df["Label_Grupo"].to_numpy(),
        tokenizer = tokenizer,
        max_len = MAX_LEN
    )
    return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)


In [15]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [16]:
# Cargamos el modelo BERT

class BERTTextClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BERTTextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
      outputs = self.bert(
          input_ids = input_ids,
          attention_mask = attention_mask
      )
      pooled_out = outputs.pooler_output
      drop_out = self.dropout(pooled_out)
      output = self.linear(drop_out)
      return output


In [17]:
model = BERTTextClassifier(NCLASS)
model = model.to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

c:\Users\ISAIAS_QUINTERO\Desktop\Trabajo Practica\venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ISAIAS_QUINTERO\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
print(model)

BERTTextClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [19]:
# ENTRENAMIENTO

EPOCHS = 5
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
total_steps = (train_data_loader.batch_size) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [20]:
# Iteracion de Entranamiento

def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids = input_ids, attention_mask = attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids = input_ids, attention_mask = attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
    return correct_predictions.double() / n_examples, np.mean(losses)

In [21]:
len(df_train)

754

In [ ]:
# Entrenamiento del Modelo

for epoch in range(EPOCHS):
  print("Epoch {} de {}".format(epoch + 1, EPOCHS))
  print("-----------------------------------")

  train_acc, train_loss = train_model(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train))

  test_acc, test_loss = eval_model(model, test_data_loader, loss_fn, device, len(df_test))

  print("Entramiento: Loss: {}, accuracy: {}".format(train_loss, train_acc))
  print("Evaluacion: Loss: {}, accuracy: {}".format(test_loss, test_acc))
  print("")


Epoch 1 de 5
-----------------------------------
